In [1]:
pip install pyspark==3.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install sparknlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pyspark
from pyspark import *
import sparknlp

In [4]:
spark = sparknlp.start()

In [5]:
train_data = spark.read.option('Header',True).csv('/content/train_data.csv')
test_data = spark.read.option('Header',True).csv('/content/test_data.csv')

In [6]:
train_data.show()

+--------------------+------+
|                data|salary|
+--------------------+------+
|5 7 yrs exp minim...|     5|
|10 17 yrs he shou...|     1|
|5 9 yrs must be a...|     2|
|7 10 yrs 7 10 yea...|     1|
|1 3 yrs chartered...|     4|
|5 10 yrs 4 willin...|     5|
|13 15 yrs experie...|     3|
|6 10 yrs qualifie...|     5|
|1 3 yrs proven ex...|     0|
|2 6 yrs other act...|     2|
|3 7 yrs manage la...|     5|
|10 12 yrs experie...|     1|
|2 6 yrs 3 years o...|     1|
|2 4 yrs knowledge...|     5|
|1 6 yrs job respo...|     1|
|3 6 yrs masters m...|     1|
|5 8 yrs 2 years o...|     5|
|3 6 yrs 3 years e...|     1|
|4 9 yrs work with...|     2|
|3 5 yrs experienc...|     5|
+--------------------+------+
only showing top 20 rows



In [7]:
test_data.show()

+--------------------+
|                data|
+--------------------+
|7 12 yrs professi...|
|0 5 yrs we are lo...|
|3 6 yrs should un...|
|0 3 yrs no data a...|
|0 5 yrs no data a...|
|4 6 yrs 5 6 years...|
|10 20 yrs no data...|
|0 2 yrs no data a...|
|6 8 yrs 6 8 years...|
|0 5 yrs no data a...|
|10 15 yrs ensure ...|
|6 9 yrs 6 years o...|
|3 7 yrs at least ...|
|8 12 yrs team man...|
|3 6 yrs should ha...|
|2 4 yrs expertise...|
|1 5 yrs should ha...|
|0 2 yrs no data a...|
|5 10 yrs no data ...|
|3 6 yrs the candi...|
+--------------------+
only showing top 20 rows



In [8]:
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline

In [9]:
document_assembler = DocumentAssembler()
sentence_encoder = UniversalSentenceEncoder.pretrained()
embeddings_finisher = EmbeddingsFinisher()

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [15]:
document_assembler.setInputCol('data')
document_assembler.setOutputCol('document')

sentence_encoder.setInputCols(['document'])
sentence_encoder.setOutputCol('sentence_embeddings')

embeddings_finisher.setInputCols("sentence_embeddings")
embeddings_finisher.setOutputCols("finished_sentence_embeddings")
embeddings_finisher.setOutputAsVector(True)
embeddings_finisher.setCleanAnnotations(True)

EmbeddingsFinisher_c925f8c7283b

In [16]:
nlp_pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_encoder,
    embeddings_finisher
])

In [17]:
pipeline_model = nlp_pipeline.fit(train_data.select('data'))
embedded_model = pipeline_model.transform(train_data.select('data'))

In [29]:
embedded_model.toPandas().to_csv('train_data_embedded.csv',index=False)

In [30]:
pipeline_model1 = nlp_pipeline.fit(test_data.select('data'))
embedded_model1 = pipeline_model1.transform(test_data.select('data'))

In [31]:
embedded_model1.toPandas().to_csv('test_data_embedded.csv',index=False)